## Свой GAN на полносвязных слоях

In [10]:
!pip3 install websocket-client torch torchvision sklearn

In [12]:
import torch
from matplotlib import pyplot as plt
from sklearn.datasets import load_digits
from torch import optim
import torch.nn.functional as F
import torch.nn as nn
from torch.utils import data
import numpy as np
from IPython.display import clear_output
from torchvision import datasets, transforms

1) Считать FashionMNIST и создать DataLoader

In [16]:
import torch
import torch.nn as nn
from torch import optim
from torch.nn import MSELoss
import torch.nn.functional as F

from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST

from sklearn.metrics import accuracy_score


IMG_SIZE = 64

ds_train = datasets.ImageFolder("/run/user/1000/gvfs/smb-share:domain=WORKGROUP,server=192.168.1.81,share=pavel,user=pavtiger/Code/NASA/perseverance/data", transform=transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
#     transforms.RandomHorizontalFlip(0.2),
#     transforms.RandomPerspective(0.2, 0.2),
    transforms.ToTensor()
]))

N = 32
train_loader = DataLoader(ds_train, batch_size=N)

2) Создайте две сети: 
- Генератор решает задачу регрессии, предсказывая по вектору шума пиксели картинки (размер шума можно сделать параметром)
- Дискриминатор по картинке предсказывает её класс, фейк\не фейк (0\1)

В целом, для адекватной генерации должно хватить трёх слоёв и там и там


In [17]:
# Задайте архитектуры

D = 10

gen = nn.Sequential(
    nn.Linear(D, IMG_SIZE**2),
    nn.LeakyReLU(0.1),
    nn.Linear(IMG_SIZE**2, IMG_SIZE**2),
    nn.LeakyReLU(0.1),
    nn.Linear(IMG_SIZE**2, 3 * IMG_SIZE**2),
    nn.Tanh()
).cuda()


disc = nn.Sequential(
    nn.Flatten(),
    nn.Linear(3 * IMG_SIZE**2, 3 * IMG_SIZE**2),
    nn.LeakyReLU(0.1),
    nn.Linear(3 * IMG_SIZE**2, 3 * IMG_SIZE**2),
    nn.LeakyReLU(0.1),
    nn.Linear(3 * IMG_SIZE**2, 1),
    nn.Sigmoid()
).cuda()

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

3) Визуализация

In [18]:
import socketio

# standard Python
sio = socketio.Client()

# asyncio
# sio = socketio.AsyncClient()

sio.connect('http://localhost:1303')
print('my sid is', sio.sid)



websocket-client package not installed, only polling transport is available


my sid is 817TjR5y1G_SAT3LAAAK


In [ ]:
# Для визуализации сгенерированных картинок во время обучения


def plot_gen(generator):
    '''
    Рисует 20 сгенерированных картинок

    Параметры:

    generator - модель генератора, которая принимает шум размером (n, d) и возвращает сгенерированные картинки размером (n, 28, 28)
    '''
    plt.figure(figsize=(15, 3))

    # Сгенерируйте 20 картинок

    # код (шум для генератора размером (20, d), где d - длина вектора шума)
    noise = torch.rand(20, D)
    # print(noise)

    # код (сгенерированные по шуму картинки)
    gener = gen(noise.cuda())

    # Рисуем 20 картинок
    for i in range(1, 21):
        # Выбираем i-й квадратик
        plt.subplot(2, 10, i)
        # Рисуем картинку
        
        sio.emit('update', )
        plt.imshow(gener[i - 1].cpu().detach().numpy().reshape(3, IMG_SIZE, IMG_SIZE))
        # Убираем координаты на осях
        plt.xticks([])
        plt.yticks([])

    plt.show()

In [ ]:
plot_gen(gen)

### Обучение

![](https://developers.google.com/machine-learning/gan/images/gan_diagram.svg)

Цикл обучения будет абсолютно таким же как и в обычном классификаторе картинок, который вы писали. 

Для удобства, давайте разделим шаг обучения дискриминатора и шаг обучения генератора в две отдельные функции, которые будут вызываться друг за другом в цикле

In [ ]:
from torch.nn import BCELoss

criterion = BCELoss()  # From now on error = criterion(pred, y)

def discriminator_train_step(discriminator, generator, opt_d, x):
    '''
    Выполняет шаг обучения дискриминатора
    1) Сгенерировать фейковые картинки
    2) Получить предсказания дискриминатора для фейковых картинок
    3) Получить предсказания дискриминатора для настоящих картинок
    4) Посчитать лосс как среднее лосса на фейковых и на настоящих картинках
    5) Обновить веса

    Параметры:

    discriminator - модель дискриминатора, которая принимает набор картинок размером (n, 28, 28) и возвращает вероятности что они фейковые (n, 1)
    generator - модель генератора, которая принимает шум размером (n, d) и возвращает сгенерированные картинки размером (n, 28, 28)
    optimizer_d - оптимизатор дискриминатора
    '''

    # D - noize size, N - number of images to generate (batch_size)

    noise = torch.rand(N, D)
    generated_images = generator(noise.cuda())

    pred_fake = discriminator(generated_images)
    pred_real = discriminator(x.cuda())

    fake_loss, real_loss = criterion(pred_fake, torch.tensor([[1.]] * N).cuda()), criterion(pred_real, torch.tensor([[0.]] * len(x)).cuda())
    # print(fake_loss, real_loss)
    
    opt_d.zero_grad()
    loss = (fake_loss + real_loss) / 2
    loss.backward()

    opt_d.step()
    opt_d.zero_grad()

    return loss.item()


Напомню, что дискриминатор минимизирует бинарную кроссэнтропийную функцию потерь для фейковых и настоящих данных:

$loss = \frac{1}{2} \Big(BCELoss(fake\_images, 1) + BCELoss(true\_images, 0)\Big)$

Тут $1$ и $0$ - векторы единиц и нулей соответствующей длины



---



In [ ]:
def generator_train_step(discriminator, generator, opt_g):
    '''
    Выполняет шаг обучения генератора
    1) Сгенерировать фейковые картинки
    2) Получить предсказания дискриминатора для фейковых картинок
    4) Посчитать лосс как минус лосс дискриминатора на фейковых картинках
    5) Обновить веса

    Параметры:

    discriminator - модель дискриминатора, которая принимает набор картинок размером (n, 28, 28) и возвращает вероятности что они фейковые (n, 1)
    generator - модель генератора, которая принимает шум размером (n, d) и возвращает сгенерированные картинки размером (n, 28, 28)
    optimizer_g - оптимизатор генератора
    '''

    noise = torch.rand(N, D)
    generated_images = generator(noise.cuda())

    pred_fake = discriminator(generated_images)

    loss = -criterion(pred_fake, torch.tensor([[1.]] * N).cuda())
    
    opt_g.zero_grad()
    loss.backward()

    opt_g.step()
    opt_g.zero_grad()

    return -loss.item()


Напомню, что генератор **максимизирует** бинарную кроссэнтропийную функцию потерь **дискриминатора** для фейковых данных:

$loss = -BCELoss(fake\_images, 1)$

Тут $1$ - векторы единиц и нулей соответствующей длины

По умолчанию лосс минимизируется, поэтому чтобы максимизировать функцию, нужно взять её с минусом. Подумайте, можно ли в данном случае обойтись без минуса? Поменять что-то другое?

(вспомните что генератор хочет обманывать дискриминатор, меняя его предсказания на неправильные)




---



**Создайте оптимизаторы для сетей (для каждой сети свой оптимизатор)**

In [ ]:
optimizer_d = optim.SGD(disc.parameters(), lr=1e-1)
optimizer_g = optim.SGD(gen.parameters(), lr=1e-1)

**Напишите цикл обучения, как в обычных нейросетях:**
- проход по эпохам
  - проход по dataloader
    - обучение генератора
    - обучение дискриминатора
  - рисование картинок:
  ```python
  clear_output(True) # очистка предыдущих картинок
  plot_gen(gen) # рисование новых на том же месте
  ```

In [ ]:
import matplotlib.pyplot as plt

g_loss_graph = []
d_loss_graph = []


for epoch in range(200):
    g_losses, d_losses = [], []

    for x, y in train_loader:
        g_loss = generator_train_step(disc, gen, optimizer_g)
        d_loss = discriminator_train_step(disc, gen, optimizer_d, x)
        
        g_losses.append(g_loss)
        d_losses.append(d_loss)

    clear_output(True) # очистка предыдущих картинок
    print(f"Epoch number: {epoch}")

    d_loss = sum(d_losses) / len(d_losses)
    g_loss = sum(g_losses) / len(g_losses)

    print(f"Discriminator loss: {d_loss}, Generator loss: {g_loss}")
    plot_gen(gen) # рисование новых на том же месте

    g_loss_graph.append(g_loss)
    d_loss_graph.append(d_loss)

    plt.plot(g_loss_graph)
    plt.ylabel('generator loss')
    plt.show()

    plt.plot(d_loss_graph)
    plt.ylabel('discriminator loss')
    plt.show()

### Визуализация результатов

In [ ]:
plot_gen(gen)